<a href="https://colab.research.google.com/github/Imtiazul-Islam/ai-models/blob/main/Text-To-Image/z-image-turbo-on-t4-on-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title # **🚀 PART 1: ENVIRONMENT SETUP**
# @markdown ### ComfyUI Installation & Model Download
# @markdown This cell sets up ComfyUI and downloads the Z-Image-Turbo FP8 models optimized for Colab T4 GPU.

import os
import subprocess
from pathlib import Path

# Configuration
WORK_DIR = Path("/content/ComfyUI")
MODEL_REPO = "ImtiazulIslam/Z-Image-Turbo-FP8"

class SetupManager:
    """Manages the installation and configuration of ComfyUI environment."""

    def __init__(self, work_dir: Path):
        self.work_dir = work_dir
        self.models = {
            "diffusion": {
                "url": f"https://huggingface.co/{MODEL_REPO}/resolve/main/z-image-turbo-fp8-e4m3fn.safetensors",
                "path": work_dir / "models/diffusion_models",
                "filename": "z-image-turbo-fp8-e4m3fn.safetensors"
            },
            "clip": {
                "url": f"https://huggingface.co/{MODEL_REPO}/resolve/main/qwen_3_4b.safetensors",
                "path": work_dir / "models/clip",
                "filename": "qwen_3_4b.safetensors"
            },
            "vae": {
                "url": f"https://huggingface.co/{MODEL_REPO}/resolve/main/ae.safetensors",
                "path": work_dir / "models/vae",
                "filename": "ae.safetensors"
            }
        }

    def clone_repository(self):
        """Clone ComfyUI repository if not exists."""
        if not self.work_dir.exists():
            print("📦 Cloning ComfyUI repository...")
            subprocess.run(
                ["git", "clone", "https://github.com/Imtiazul-Islam/ComfyUI", str(self.work_dir)],
                check=True, stdout=subprocess.DEVNULL
            )
            print("✅ Repository cloned successfully")
        else:
            print("✅ ComfyUI already exists")

    def install_dependencies(self):
        """Install required Python packages and system dependencies."""
        print("📚 Installing dependencies...")
        os.chdir(self.work_dir)

        # Install Python requirements
        subprocess.run(
            ["pip", "install", "-q", "-r", "requirements.txt"],
            check=True
        )

        # Install aria2 for fast downloads
        subprocess.run(
            ["apt-get", "-y", "install", "-qq", "aria2"],
            check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
        )
        print("✅ Dependencies installed")

    def download_model(self, model_type: str):
        """Download a specific model using aria2."""
        model_info = self.models[model_type]
        model_info["path"].mkdir(parents=True, exist_ok=True)

        output_file = model_info["path"] / model_info["filename"]

        if output_file.exists():
            print(f"⏭️  {model_type.upper()} model already exists")
            return

        print(f"⬇️  Downloading {model_type.upper()} model...")
        subprocess.run([
            "aria2c",
            "--console-log-level=error",
            "-c", "-x", "16", "-s", "16", "-k", "1M",
            model_info["url"],
            "-d", str(model_info["path"]),
            "-o", model_info["filename"]
        ], check=True)
        print(f"✅ {model_type.upper()} model downloaded")

    def setup(self):
        """Run complete setup process."""
        print("=" * 60)
        print("🎨 Z-IMAGE-TURBO SETUP")
        print("=" * 60)

        self.clone_repository()
        self.install_dependencies()

        print("\n📥 Downloading models...")
        for model_type in self.models.keys():
            self.download_model(model_type)

        print("\n" + "=" * 60)
        print("✨ Setup complete! Ready to generate images.")
        print("=" * 60)

# Execute setup
setup_manager = SetupManager(WORK_DIR)
setup_manager.setup()

In [ ]:
# @title # **🎨 PART 2: IMAGE GENERATION INTERFACE**
# @markdown ### Interactive UI for Z-Image-Turbo
# @markdown Configure your generation parameters and create high-quality images with the optimized FP8 model.

import os
import sys
import random
import time
import datetime
from pathlib import Path
from typing import Tuple, Optional

import torch
import numpy as np
from PIL import Image
from IPython.display import display, clear_output, HTML as DisplayHTML
import ipywidgets as widgets

# Setup environment
os.chdir(WORK_DIR)
sys.path.insert(0, str(WORK_DIR))

# Import ComfyUI components
from nodes import NODE_CLASS_MAPPINGS

# ============================================================
# CUSTOM CSS STYLES
# ============================================================

CUSTOM_CSS = """
<style>
    /* Hide default Jupyter styling conflicts */
    .widget-html-content h3 { margin: 0 0 12px 0 !important; }

    /* Main container styling */
    .zt-container {
        font-family: 'Segoe UI', system-ui, -apple-system, sans-serif;
        max-width: 1400px;
        margin: 0 auto;
    }

    /* Header styling */
    .zt-header {
        background: linear-gradient(135deg, #1a1a2e 0%, #16213e 50%, #0f3460 100%);
        border-radius: 16px;
        padding: 30px;
        margin-bottom: 24px;
        box-shadow: 0 10px 40px rgba(0,0,0,0.3);
        position: relative;
        overflow: hidden;
    }

    .zt-header::before {
        content: '';
        position: absolute;
        top: -50%;
        left: -50%;
        width: 200%;
        height: 200%;
        background: radial-gradient(circle, rgba(255,255,255,0.1) 0%, transparent 70%);
        animation: pulse 4s ease-in-out infinite;
    }

    @keyframes pulse {
        0%, 100% { transform: scale(1); opacity: 0.5; }
        50% { transform: scale(1.1); opacity: 0.3; }
    }

    .zt-header h1 {
        color: #ffffff;
        font-size: 2.2em;
        font-weight: 700;
        margin: 0;
        text-align: center;
        text-shadow: 0 2px 10px rgba(0,0,0,0.3);
        position: relative;
        z-index: 1;
    }

    .zt-header .subtitle {
        color: #a0aec0;
        text-align: center;
        margin-top: 8px;
        font-size: 1.1em;
        position: relative;
        z-index: 1;
    }

    .zt-header .badge-container {
        display: flex;
        justify-content: center;
        gap: 12px;
        margin-top: 16px;
        position: relative;
        z-index: 1;
    }

    .zt-badge {
        background: rgba(255,255,255,0.15);
        color: #fff;
        padding: 6px 14px;
        border-radius: 20px;
        font-size: 0.85em;
        backdrop-filter: blur(10px);
        border: 1px solid rgba(255,255,255,0.2);
    }

    /* Card styling */
    .zt-card {
        background: #ffffff;
        border-radius: 12px;
        padding: 20px;
        margin-bottom: 16px;
        box-shadow: 0 2px 12px rgba(0,0,0,0.08);
        border: 1px solid #e2e8f0;
        transition: all 0.3s ease;
    }

    .zt-card:hover {
        box-shadow: 0 4px 20px rgba(0,0,0,0.12);
        transform: translateY(-2px);
    }

    .zt-card-dark {
        background: linear-gradient(145deg, #1e1e2e, #2d2d3f);
        border: 1px solid #3d3d5c;
    }

    .zt-card-dark:hover {
        border-color: #5a5a8a;
    }

    .zt-card-title {
        font-size: 1.1em;
        font-weight: 600;
        color: #2d3748;
        margin-bottom: 16px;
        padding-bottom: 10px;
        border-bottom: 2px solid #e2e8f0;
        display: flex;
        align-items: center;
        gap: 8px;
    }

    .zt-card-dark .zt-card-title {
        color: #e2e8f0;
        border-bottom-color: #3d3d5c;
    }

    /* Output area styling */
    .zt-output-area {
        background: linear-gradient(145deg, #fafafa, #f0f0f0);
        border-radius: 12px;
        padding: 20px;
        min-height: 200px;
        border: 2px dashed #cbd5e0;
        display: flex;
        flex-direction: column;
        align-items: center;
        justify-content: center;
    }

    .zt-output-placeholder {
        color: #a0aec0;
        font-size: 1.1em;
        text-align: center;
    }

    .zt-output-placeholder .icon {
        font-size: 3em;
        margin-bottom: 12px;
        opacity: 0.5;
    }

    /* Status messages */
    .zt-status {
        padding: 12px 16px;
        border-radius: 8px;
        margin-bottom: 12px;
        font-size: 0.95em;
    }

    .zt-status-info {
        background: linear-gradient(135deg, #e3f2fd, #bbdefb);
        color: #1565c0;
        border-left: 4px solid #1976d2;
    }

    .zt-status-success {
        background: linear-gradient(135deg, #e8f5e9, #c8e6c9);
        color: #2e7d32;
        border-left: 4px solid #43a047;
    }

    .zt-status-generating {
        background: linear-gradient(135deg, #fff3e0, #ffe0b2);
        color: #e65100;
        border-left: 4px solid #fb8c00;
    }

    /* Image result styling */
    .zt-image-result {
        background: #ffffff;
        border-radius: 12px;
        padding: 16px;
        box-shadow: 0 4px 20px rgba(0,0,0,0.15);
        margin-top: 16px;
    }

    .zt-image-result img {
        border-radius: 8px;
        max-width: 100%;
        display: block;
        margin: 0 auto;
    }

    .zt-image-info {
        display: grid;
        grid-template-columns: repeat(auto-fit, minmax(150px, 1fr));
        gap: 12px;
        margin-top: 16px;
        padding-top: 16px;
        border-top: 1px solid #e2e8f0;
    }

    .zt-info-item {
        background: #f7fafc;
        padding: 10px 14px;
        border-radius: 8px;
        text-align: center;
    }

    .zt-info-item .label {
        font-size: 0.8em;
        color: #718096;
        text-transform: uppercase;
        letter-spacing: 0.5px;
    }

    .zt-info-item .value {
        font-size: 1.1em;
        color: #2d3748;
        font-weight: 600;
        margin-top: 4px;
    }

    /* Tips section */
    .zt-tips {
        background: linear-gradient(135deg, #f0f9ff, #e0f2fe);
        border-radius: 10px;
        padding: 16px;
        border-left: 4px solid #0284c7;
    }

    .zt-tips h4 {
        color: #0369a1;
        margin: 0 0 10px 0;
        font-size: 0.95em;
    }

    .zt-tips ul {
        margin: 0;
        padding-left: 20px;
        color: #0c4a6e;
        font-size: 0.9em;
    }

    .zt-tips li {
        margin-bottom: 6px;
    }

    /* Responsive layout */
    .zt-main-layout {
        display: flex;
        gap: 24px;
    }

    .zt-left-column {
        flex: 1.2;
        min-width: 0;
    }

    .zt-right-column {
        flex: 0.8;
        min-width: 320px;
    }

    /* Footer */
    .zt-footer {
        text-align: center;
        padding: 16px;
        color: #718096;
        font-size: 0.85em;
        margin-top: 20px;
    }

    .zt-footer a {
        color: #4299e1;
        text-decoration: none;
    }
</style>
"""

# Inject CSS
display(DisplayHTML(CUSTOM_CSS))

# ============================================================
# MODEL MANAGER CLASS
# ============================================================

class ModelManager:
    """Handles loading and caching of AI models."""

    def __init__(self):
        self.unet = None
        self.clip = None
        self.vae = None
        self.nodes = self._initialize_nodes()

    def _initialize_nodes(self):
        """Initialize ComfyUI node instances."""
        return {
            'unet_loader': NODE_CLASS_MAPPINGS["UNETLoader"](),
            'clip_loader': NODE_CLASS_MAPPINGS["CLIPLoader"](),
            'vae_loader': NODE_CLASS_MAPPINGS["VAELoader"](),
            'clip_encode': NODE_CLASS_MAPPINGS["CLIPTextEncode"](),
            'sampler': NODE_CLASS_MAPPINGS["KSampler"](),
            'vae_decode': NODE_CLASS_MAPPINGS["VAEDecode"](),
            'empty_latent': NODE_CLASS_MAPPINGS["EmptyLatentImage"]()
        }

    def load_models(self):
        """Load all required models into VRAM."""
        with torch.inference_mode():
            self.unet = self.nodes['unet_loader'].load_unet(
                "z-image-turbo-fp8-e4m3fn.safetensors",
                "fp8_e4m3fn_fast"
            )[0]

            self.clip = self.nodes['clip_loader'].load_clip(
                "qwen_3_4b.safetensors",
                type="lumina2"
            )[0]

            self.vae = self.nodes['vae_loader'].load_vae("ae.safetensors")[0]

        return self.nodes

# ============================================================
# IMAGE GENERATOR CLASS
# ============================================================

class ImageGenerator:
    """Handles the image generation pipeline."""

    def __init__(self, model_manager: ModelManager, output_dir: Path):
        self.model_manager = model_manager
        self.output_dir = output_dir
        self.output_dir.mkdir(exist_ok=True)

    def generate_seed(self, seed: int) -> int:
        """Generate or validate seed value."""
        if seed == 0:
            random.seed(int(time.time()))
            return random.randint(0, 2**64 - 1)
        return seed

    @torch.inference_mode()
    def generate(
        self,
        positive_prompt: str,
        negative_prompt: str,
        width: int,
        height: int,
        steps: int,
        cfg: float,
        seed: int,
        sampler: str,
        scheduler: str,
        denoise: float
    ) -> Tuple[Image.Image, int, str]:
        """Generate an image based on the provided parameters."""
        nodes = self.model_manager.nodes
        actual_seed = self.generate_seed(seed)

        # Encode prompts
        positive_cond = nodes['clip_encode'].encode(
            self.model_manager.clip, positive_prompt
        )[0]
        negative_cond = nodes['clip_encode'].encode(
            self.model_manager.clip, negative_prompt
        )[0]

        # Create latent
        latent = nodes['empty_latent'].generate(width, height, batch_size=1)[0]

        # Sample
        samples = nodes['sampler'].sample(
            self.model_manager.unet,
            actual_seed,
            steps,
            cfg,
            sampler,
            scheduler,
            positive_cond,
            negative_cond,
            latent,
            denoise=denoise
        )[0]

        # Decode
        decoded = nodes['vae_decode'].decode(self.model_manager.vae, samples)[0].detach()

        # Convert to PIL
        img_array = np.array(decoded * 255, dtype=np.uint8)[0]
        image = Image.fromarray(img_array)

        # Save
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"z_turbo_{timestamp}_s{actual_seed}.png"
        output_path = self.output_dir / filename
        image.save(output_path)

        return image, actual_seed, str(output_path)

# ============================================================
# GENERATION UI CLASS
# ============================================================

class GenerationUI:
    """Creates and manages the professional UI."""

    ASPECT_RATIOS = {
        "1:1 Square": (1024, 1024),
        "16:9 Landscape": (1344, 768),
        "9:16 Portrait": (768, 1344),
        "4:3 Standard": (1152, 896),
        "3:2 Classic": (1216, 832),
        "21:9 Cinematic": (1536, 640),
        "Custom": None
    }

    SAMPLERS = ['euler', 'euler_ancestral', 'dpmpp_2m', 'dpmpp_sde']
    SCHEDULERS = ['simple', 'normal', 'karras', 'exponential', 'sgm_uniform']

    def __init__(self, generator: ImageGenerator):
        self.generator = generator
        self.widgets = {}
        self._create_widgets()
        self._setup_callbacks()

    def _create_widgets(self):
        """Create all UI widgets with professional styling."""

        # ===== PROMPT WIDGETS =====
        self.widgets['positive'] = widgets.Textarea(
            value="a cinematic photo of a majestic lion in golden savanna, dramatic lighting, "
                  "professional wildlife photography, sharp focus, National Geographic style, 8K",
            placeholder='Describe the image you want to create...',
            layout=widgets.Layout(width='100%', height='100px')
        )

        self.widgets['negative'] = widgets.Textarea(
            value="blurry, ugly, bad quality, distorted, low resolution, watermark, text, "
                  "oversaturated, underexposed, artifacts, noise",
            placeholder='Describe what to avoid...',
            layout=widgets.Layout(width='100%', height='70px')
        )

        # ===== IMAGE SETTINGS =====
        self.widgets['aspect_ratio'] = widgets.Dropdown(
            options=list(self.ASPECT_RATIOS.keys()),
            value="1:1 Square",
            layout=widgets.Layout(width='100%')
        )

        self.widgets['width'] = widgets.BoundedIntText(
            value=1024, min=512, max=2048, step=64,
            layout=widgets.Layout(width='48%')
        )

        self.widgets['height'] = widgets.BoundedIntText(
            value=1024, min=512, max=2048, step=64,
            layout=widgets.Layout(width='48%')
        )

        # ===== SAMPLING SETTINGS =====
        self.widgets['steps'] = widgets.IntSlider(
            value=6, min=1, max=20, step=1,
            description='',
            continuous_update=False,
            layout=widgets.Layout(width='100%'),
            style={'handle_color': '#4299e1'}
        )

        self.widgets['cfg'] = widgets.FloatSlider(
            value=1.2, min=1.0, max=3.0, step=0.1,
            description='',
            continuous_update=False,
            readout_format='.1f',
            layout=widgets.Layout(width='100%'),
            style={'handle_color': '#48bb78'}
        )

        self.widgets['seed'] = widgets.IntText(
            value=0,
            layout=widgets.Layout(width='100%')
        )

        self.widgets['sampler'] = widgets.Dropdown(
            options=self.SAMPLERS,
            value='euler',
            layout=widgets.Layout(width='100%')
        )

        self.widgets['scheduler'] = widgets.Dropdown(
            options=self.SCHEDULERS,
            value='simple',
            layout=widgets.Layout(width='100%')
        )

        self.widgets['denoise'] = widgets.FloatSlider(
            value=1.0, min=0.0, max=1.0, step=0.05,
            description='',
            continuous_update=False,
            readout_format='.2f',
            layout=widgets.Layout(width='100%'),
            style={'handle_color': '#ed8936'}
        )

        # ===== GENERATE BUTTON =====
        self.widgets['generate_btn'] = widgets.Button(
            description='✨ Generate Image',
            button_style='primary',
            tooltip='Click to generate your image',
            layout=widgets.Layout(width='100%', height='50px'),
            style={'font_weight': 'bold'}
        )

        # ===== OUTPUT AREA =====
        self.widgets['output'] = widgets.Output(
            layout=widgets.Layout(width='100%', min_height='300px')
        )

    def _setup_callbacks(self):
        """Setup widget callbacks."""
        self.widgets['aspect_ratio'].observe(self._update_dimensions, names='value')
        self.widgets['generate_btn'].on_click(self._on_generate)

    def _update_dimensions(self, change):
        """Update dimensions based on aspect ratio."""
        dimensions = self.ASPECT_RATIOS.get(change['new'])
        if dimensions:
            self.widgets['width'].value = dimensions[0]
            self.widgets['height'].value = dimensions[1]

    def _on_generate(self, button):
        """Handle generation with beautiful output."""
        button.disabled = True
        button.description = "⏳ Creating Magic..."
        button.button_style = 'warning'

        self.widgets['output'].clear_output(wait=True)

        with self.widgets['output']:
            # Show generating status
            display(DisplayHTML("""
                <div class="zt-status zt-status-generating">
                    <strong>🎨 Generating your masterpiece...</strong><br>
                    <small>This typically takes 10-30 seconds depending on settings</small>
                </div>
            """))

            start_time = time.time()

            try:
                image, actual_seed, output_path = self.generator.generate(
                    positive_prompt=self.widgets['positive'].value,
                    negative_prompt=self.widgets['negative'].value,
                    width=self.widgets['width'].value,
                    height=self.widgets['height'].value,
                    steps=self.widgets['steps'].value,
                    cfg=self.widgets['cfg'].value,
                    seed=self.widgets['seed'].value,
                    sampler=self.widgets['sampler'].value,
                    scheduler=self.widgets['scheduler'].value,
                    denoise=self.widgets['denoise'].value
                )

                generation_time = time.time() - start_time

                # Clear and show results
                self.widgets['output'].clear_output(wait=True)

                with self.widgets['output']:
                    # Success message
                    display(DisplayHTML(f"""
                        <div class="zt-status zt-status-success">
                            <strong>✅ Image generated successfully!</strong>
                            <span style="float: right;">⏱️ {generation_time:.1f}s</span>
                        </div>
                    """))

                    # Display image
                    display(image)

                    # Image info grid
                    display(DisplayHTML(f"""
                        <div class="zt-image-info">
                            <div class="zt-info-item">
                                <div class="label">Resolution</div>
                                <div class="value">{self.widgets['width'].value}×{self.widgets['height'].value}</div>
                            </div>
                            <div class="zt-info-item">
                                <div class="label">Seed</div>
                                <div class="value">{actual_seed}</div>
                            </div>
                            <div class="zt-info-item">
                                <div class="label">Steps</div>
                                <div class="value">{self.widgets['steps'].value}</div>
                            </div>
                            <div class="zt-info-item">
                                <div class="label">CFG Scale</div>
                                <div class="value">{self.widgets['cfg'].value}</div>
                            </div>
                            <div class="zt-info-item">
                                <div class="label">Sampler</div>
                                <div class="value">{self.widgets['sampler'].value}</div>
                            </div>
                            <div class="zt-info-item">
                                <div class="label">Time</div>
                                <div class="value">{generation_time:.1f}s</div>
                            </div>
                        </div>
                        <div style="text-align: center; margin-top: 12px; color: #718096; font-size: 0.85em;">
                            💾 Saved to: <code>{output_path}</code>
                        </div>
                    """))

            except Exception as e:
                self.widgets['output'].clear_output(wait=True)
                with self.widgets['output']:
                    display(DisplayHTML(f"""
                        <div class="zt-status" style="background: #fed7d7; color: #c53030; border-left: 4px solid #e53e3e;">
                            <strong>❌ Generation Failed</strong><br>
                            <small>{str(e)}</small>
                        </div>
                    """))
                    import traceback
                    print(traceback.format_exc())

        # Reset button
        button.disabled = False
        button.description = "✨ Generate Image"
        button.button_style = 'primary'

    def _create_labeled_widget(self, label: str, widget, icon: str = ""):
        """Create a widget with a label above it."""
        label_html = widgets.HTML(
            f'<div style="font-size: 0.9em; font-weight: 600; color: #4a5568; margin-bottom: 6px;">'
            f'{icon} {label}</div>'
        )
        return widgets.VBox([label_html, widget], layout=widgets.Layout(margin='0 0 12px 0'))

    def display(self):
        """Display the complete professional UI."""

        # ===== HEADER =====
        header = widgets.HTML("""
            <div class="zt-header">
                <h1>⚡ Z-Image-Turbo</h1>
                <p class="subtitle">Next-Generation AI Image Synthesis</p>
                <div class="badge-container">
                    <span class="zt-badge">🚀 FP8 Optimized</span>
                    <span class="zt-badge">⚡ 4-6 Steps</span>
                    <span class="zt-badge">🎯 T4 GPU Ready</span>
                </div>
            </div>
        """)

        # ===== PROMPTS CARD =====
        prompts_card = widgets.VBox([
            widgets.HTML('<div class="zt-card-title">📝 Prompt Engineering</div>'),
            self._create_labeled_widget("Positive Prompt", self.widgets['positive'], "✨"),
            self._create_labeled_widget("Negative Prompt", self.widgets['negative'], "🚫"),
        ], layout=widgets.Layout(padding='20px'))

        prompts_wrapper = widgets.HTML("""<div class="zt-card">""")

        # ===== GENERATE BUTTON CARD =====
        generate_section = widgets.VBox([
            self.widgets['generate_btn'],
        ], layout=widgets.Layout(margin='16px 0'))

        # ===== OUTPUT CARD =====
        output_card = widgets.VBox([
            widgets.HTML('<div class="zt-card-title">🖼️ Generated Output</div>'),
            self.widgets['output']
        ], layout=widgets.Layout(padding='20px'))

        # ===== IMAGE SETTINGS CARD =====
        dimension_row = widgets.HBox([
            self._create_labeled_widget("Width", self.widgets['width'], "↔️"),
            self._create_labeled_widget("Height", self.widgets['height'], "↕️"),
        ], layout=widgets.Layout(justify_content='space-between'))

        image_settings = widgets.VBox([
            widgets.HTML('<div class="zt-card-title">🖼️ Image Settings</div>'),
            self._create_labeled_widget("Aspect Ratio", self.widgets['aspect_ratio'], "📐"),
            dimension_row,
        ], layout=widgets.Layout(padding='20px'))

        # ===== SAMPLING SETTINGS CARD =====
        sampling_settings = widgets.VBox([
            widgets.HTML('<div class="zt-card-title">⚙️ Sampling Configuration</div>'),
            self._create_labeled_widget("Steps (4-8 recommended)", self.widgets['steps'], "🔢"),
            self._create_labeled_widget("CFG Scale", self.widgets['cfg'], "🎛️"),
            self._create_labeled_widget("Seed (0 = random)", self.widgets['seed'], "🎲"),
            self._create_labeled_widget("Sampler", self.widgets['sampler'], "🔧"),
            self._create_labeled_widget("Scheduler", self.widgets['scheduler'], "📊"),
            self._create_labeled_widget("Denoise Strength", self.widgets['denoise'], "🎚️"),
        ], layout=widgets.Layout(padding='20px'))

        # ===== TIPS SECTION =====
        tips = widgets.HTML("""
            <div class="zt-tips">
                <h4>💡 Pro Tips for Best Results</h4>
                <ul>
                    <li><strong>Steps:</strong> Z-Turbo excels at 4-8 steps. More isn't always better!</li>
                    <li><strong>CFG Scale:</strong> Keep between 1.0-1.5 for natural, balanced results</li>
                    <li><strong>Prompts:</strong> Be descriptive - include style, lighting, and quality terms</li>
                    <li><strong>Seed:</strong> Found a great image? Save the seed to recreate it!</li>
                </ul>
            </div>
        """)

        # ===== LEFT COLUMN =====
        left_column = widgets.VBox([
            prompts_card,
            generate_section,
            output_card,
        ], layout=widgets.Layout(
            flex='1.3',
            min_width='500px',
            padding='0 12px 0 0'
        ))

        # ===== RIGHT COLUMN =====
        right_column = widgets.VBox([
            image_settings,
            sampling_settings,
            tips,
        ], layout=widgets.Layout(
            flex='0.7',
            min_width='320px',
            padding='0 0 0 12px'
        ))

        # ===== MAIN LAYOUT =====
        main_content = widgets.HBox([
            left_column,
            right_column
        ], layout=widgets.Layout(
            display='flex',
            flex_flow='row wrap',
            justify_content='space-between'
        ))

        # ===== FOOTER =====
        footer = widgets.HTML("""
            <div class="zt-footer">
                <p>
                    🎨 Z-Image-Turbo | FP8 Optimized for Colab T4 |
                    Model by <a href="https://huggingface.co/ImtiazulIslam" target="_blank">ImtiazulIslam</a>
                </p>
            </div>
        """)

        # ===== FINAL ASSEMBLY =====
        ui = widgets.VBox([
            header,
            main_content,
            footer,
        ], layout=widgets.Layout(max_width='1400px', margin='0 auto'))

        # Show placeholder in output
        with self.widgets['output']:
            display(DisplayHTML("""
                <div class="zt-output-area">
                    <div class="zt-output-placeholder">
                        <div class="icon">🎨</div>
                        <div>Your generated image will appear here</div>
                        <div style="font-size: 0.85em; margin-top: 8px; opacity: 0.7;">
                            Configure your settings and click "Generate Image"
                        </div>
                    </div>
                </div>
            """))

        display(ui)

# ============================================================
# INITIALIZE AND LAUNCH
# ============================================================

print("⚡ Initializing Z-Image-Turbo Generator...")
print("━" * 50)

# Load models
print("📦 Loading UNET model...")
model_manager = ModelManager()
print("📦 Loading CLIP encoder...")
print("📦 Loading VAE decoder...")
model_manager.load_models()
print("✅ All models loaded successfully!")
print("━" * 50)
print()

# Create generator and UI
generator = ImageGenerator(
    model_manager=model_manager,
    output_dir=WORK_DIR / "output"
)

ui = GenerationUI(generator)
ui.display()